In [1]:
pip install transformers


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install torch -i https://pypi.org/simple

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

def predict_sentiment_batch(texts, batch_size=32):
    sentiments = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
            return_token_type_ids=False
        )
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        labels = torch.argmax(probs, dim=1).tolist()
        sentiments.extend(["negatif", "netral", "positif"][label] for label in labels)
    return sentiments

df = pd.read_csv("YT_comments_IKN.csv")
df["Sentiment"] = predict_sentiment_batch(df["Comment"].tolist(), batch_size=32)

print(df[["Comment", "Sentiment"]].head(10))
df.to_csv("YT_comments_labelled.csv", index=False)
print("✅ File berhasil disimpan: YT_comments_labelled.csv")


c:\Users\Asus Gk\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                             Comment Sentiment
0  Yang bilang saya menjilat.. lha wong kenyataaa...   positif
1  Berarti Indonesia tidak miskin seperti dugaan ...   positif
2  😂😂😂😂😂😂 rata rata yg komen disini pendukung Mul...   positif
3  Orang yng mengatakan IKN  mangkrak, orang itu ...   positif
4      IKN..indonesia ku semakin jaya amin ya Allahu   negatif
5  Yang bilang ikn mangkrak itu manusia yang kaya...   positif
6         gasama amin rais mas.😂 keruangam mewahnya,   negatif
7                                   IKN memang keren   negatif
8                 Soto kok gak blh make kecap knp om   positif
9  Surabaya hadir. Alhamdulillah. Terimakasih Cin...    netral
✅ File berhasil disimpan: YT_comments_labelled.csv


In [7]:
from tabulate import tabulate
import pandas as pd

# Fungsi potong komentar
def potong_komentar(teks, max_kata=20):
    kata = teks.split()
    if len(kata) > max_kata:
        return ' '.join(kata[:max_kata]) + "..."
    return teks

# Baca file hasil label
df = pd.read_csv("YT_comments_labelled.csv")

# Potong komentar panjang
df["ShortComment"] = df["Comment"].apply(lambda x: potong_komentar(str(x), 20))

# Tampilkan komentar singkat dan label (maks 10 baris)
print(tabulate(df[["ShortComment", "Sentiment"]].head(10), headers="keys", tablefmt="grid"))


+----+---------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|    | ShortComment                                                                                                                                | Sentiment   |
+====+=============================================================================================================================================+=============+
|  0 | Yang bilang saya menjilat.. lha wong kenyataaannya memang ada pembangunan.. kalo mangkrak itu ditinggalkan dan tidak ada yg kerja.. saya... | positif     |
+----+---------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|  1 | Berarti Indonesia tidak miskin seperti dugaan orang malaydes😅                                                                              | positif     |
+----+-----------------